In [1]:
import os
import pandas as pd
import numpy as np
import geocoder
import feather
import re
from collections import Counter
import mojimoji
import geocoder

In [2]:
train = pd.read_csv('../input/train.csv')
target = train["賃料"]
train = train.drop("賃料", axis=1)
test = pd.read_csv('../input/test.csv')

train_length = train.shape[0]
test_length = test.shape[0]
all_df = pd.concat([train, test], axis=0, ignore_index=True)

In [3]:
def calculate_meter(values):
    """list から 距離情報が含まれている要素に対して演算"""
    tmp = values
    tmp_meter = 0
    for t in range(len(tmp)):
        kyori = 0
        if "徒歩" in tmp[t]:
            tmp[t] = mojimoji.zen_to_han(tmp[t], ascii=False)
            try:
                kyori = int(re.sub("[^0-9]", "", tmp[t].split("徒歩")[-1]))*80
            except:
                kyori = 0
            tmp_meter += kyori
        elif "バス" in tmp[t]:
            tmp[t] = mojimoji.zen_to_han(tmp[t], ascii=False)
            try:
                kyori = int(re.sub("[^0-9]", "", tmp[t]))*400
            except:
                kyori = 0
            tmp_meter += kyori
        elif "km" in tmp[t]:
            tmp[t] = mojimoji.zen_to_han(tmp[t], ascii=False)
            try:
                kyori = int(float(re.sub("[^0-9.]", "", tmp[t].split("km")[0]))*1000)
            except:
                kyori = 0
            tmp_meter += kyori
    return tmp_meter

In [4]:
tmp_df = all_df["アクセス"].str.split("\t\t", expand=True, n=2).fillna("NaN")
i = 0
"""i==0のときはこれで全部行ける"""
df_out = pd.DataFrame()
railway = []
station = []
minute = []
for values in tmp_df[i]:
    railway.append(values.split("\t")[0])
    station.append(values.split("\t")[1])
    tmp = re.findall(".*?[0-9]分", values.split("\t")[2])
    minute.append(calculate_meter(tmp))

df_out[f"路線_{i}"] = railway
df_out[f"駅_{i}"] = station
df_out[f"駅からの距離_{i}"] = minute

all_railways = list(set(df_out["路線_0"]))
all_stations = list(set(df_out["駅_0"]))


"""長さ順にソート + 新規の路線、駅追加"""
#all_stations.append() 港町駅
new_station_i1 = ['有楽町駅', '鹿島田駅', '市川駅', '舞浜駅', '産業道路駅', '都電雑司ケ谷駅', '三田(兵庫県)駅', #df完成後に三田（兵庫県）変えようね
                  '東京テレポート駅', '荒川七丁目駅', '新木場駅', '竹芝駅', '松戸駅', '国府台駅', '妙典駅', '飛鳥山駅', 
                  '日暮里(舎人ライナー)駅', 'ひばりケ丘(東京都)駅', '川口駅', '国会議事堂前駅', '鈴木町駅', '井の頭公園駅', 
                  '新庚申塚駅', '霞ケ関(東京都)駅', '荒川遊園地前駅', 'つつじケ丘駅', '庚申塚駅', '二子新地駅', '内幸町駅', 
                  '東京ディズニーランド・ステーション駅', '南行徳駅', "港町駅", "有明(東京都)駅", "三鷹駅", "犇", "市場前駅",
                  "新丸子駅", "東門前駅", "平間駅", "流通センター駅", "三郷中央駅", "荒川二丁目駅", "西武柳沢駅",
                  "南鳩ケ谷駅", "本八幡(総武線)駅", "高津(神奈川県)駅", "川口元郷駅", "市川真間駅", "志木駅", "行徳駅", "台場駅"
                  ]

new_railway_i1 = ["京急大師線", "ディズニーリゾートライン", "南武線", "三田線", "南武線(川崎－立川)", "犇"]
[all_stations.append(new_eki) for new_eki in new_station_i1]
tmp = []
for i in range(len(all_stations)):
    tmp.append(list((len(all_stations[i]), all_stations[i])))
all_stations = sorted(tmp, reverse=True)

[all_railways.append(new_path) for new_path in new_railway_i1]
tmp = []
for i in range(len(all_railways)):
    tmp.append(list((len(all_railways[i]), all_railways[i])))
all_railways = sorted(tmp, reverse=True)

In [5]:
"""i=1の時　専用"""
for i in range(1,tmp_df.shape[1]-1):
    railway = []
    station = []
    minute = []
    search_len = []
    new_station = []
    for values in tmp_df[i]:
        p=0
        values = re.sub("[:：]", "\t", values)
        values = re.sub("[\s]", "", values)
        values = re.sub("\u3000", "\t", values)
        
        if values == "NaN":
            railway.append("NaN")
            station.append("NaN")
            minute.append(np.nan)
            search_len.append("NaN")

        elif values != "NaN":
            rail_flag = False
            #i>1では書き方が変わって　\t　では分割できない時もある"""
            for path in all_railways:
                if path[1] in values and rail_flag == False:
                    railway.append(path[1])
                    rail_flag = True
                    break
                    
                if path[1] == all_railways[-1][1]:
                    if rail_flag == False:
                        railway.append("NaN")
                        rail_flag = True
                        break
            
            """以下のfor文は「path[1]が見つかった場合のみ回る」、values=小76／松本橋東詰6分　の時はstationに何も加えない"""
            sta_flag = False
            for eki in all_stations:
                for val in values.split(path[1],1)[1:]:
                    if eki[1] in val and sta_flag == False:
                        station.append(eki[1])
                        sta_flag = True
                        break
                        
                    if eki[1] == all_stations[-1][1]:
                        if sta_flag == False:   #Trueにしたら新規の駅が除かれる
                            station.append("NaN")
                            sta_flag = True
                            break
                if sta_flag:
                    break
            

            
            
            if len(values.split(eki[1])[1:])==0:   # 駅や路線が新たに追加された時用  #小76／松本橋東詰6分 三田線　未処理
                try:
                    print(values, eki[1], path[1] , "  ", values.split(path[1],1)[1].split("駅")[0]+"駅")
                    new_station.append(values.split(path[1],1)[1].split("駅")[0]+"駅")
                except:
                    print(values)
            search_len.append(len(values.split(eki[1])))

            
            try:
                if "km" not in values.split(path[1],1)[1].split(eki[1],1)[1] and "分" not in values.split(path[1],1)[1].split(eki[1],1)[1]:
                    minute.append(np.nan)
                    print(values)
                else:    
                    meter_infos = re.split("分", values.split(path[1],1)[1].split(eki[1],1)[1])
                    minute.append(calculate_meter(meter_infos))
            
            except:
                #print(values, eki[1], path[1])
                if sta_flag == False:
                    station.append("NaN")
                minute.append(np.nan)
            
            """if values == "小76／松本橋東詰6分":
                station.append(1)
                railway.append(2)"""

        
    df_out[f"路線_{i}"] = railway
    df_out[f"駅_{i}"] = station
    df_out[f"駅からの距離_{i}"] = minute
    print(f"{i}_done")

小76／松本橋東詰6分
1_done


In [6]:
"""i=2の時　専用"""
for i in range(1,tmp_df.shape[1]):
    railway = []
    station = []
    minute = []
    search_len = []
    new_station = []
    for values in tmp_df[i]:
        values = values.split("\t\t")[0]
        values = re.sub("[:：]", "\t", values)
        values = re.sub("[\s]", "", values)
        values = re.sub("\u3000", "\t", values)
        
        values = re.sub("[「」『』]", "", values)
        
        if values == "NaN":
            railway.append("NaN")
            station.append("NaN")
            minute.append(np.nan)
            search_len.append("NaN")

        elif values != "NaN":
            rail_flag = False
            #i>1では書き方が変わって　\t　では分割できない時もある"""
            for path in all_railways:
                if path[1] in values and rail_flag == False:
                    railway.append(path[1])
                    rail_flag = True
                    break
                    
                if path[1] == all_railways[-1][1]:
                    if rail_flag == False:
                        if "京葉線" in values:
                            path[1] = "京葉線"
                            railway.append("京葉線(東京－蘇我)")
                        elif "中央・総武緩行線" in values:
                            path[1] = "中央・総武緩行線" 
                            railway.append("総武線・中央線（各停）")
                        elif "小田急電鉄小田原線" in values:
                            path[1] = "小田急電鉄小田原線" 
                            railway.append("小田急小田原線")
                        elif "西武池袋・豊島線" in values:
                            path[1] = "西武池袋・豊島線"
                            railway.append("西武池袋線")
                        elif "中央本線" in values:
                            path[1] = "中央本線"
                            railway.append('中央本線(東京－松本)')
                        elif "京王電鉄井の頭線" in values:
                            path[1] = "京王電鉄井の頭線"
                            railway.append('京王井の頭線')
                        elif "東京臨海高速鉄道" in values:
                            path[1] = "東京臨海高速鉄道"
                            railway.append('りんかい線')
                        elif "京浜東北・根岸線" in values:
                            path[1] = "京浜東北・根岸線"
                            railway.append('京浜東北線')
                        
                        else:
                            railway.append("NaN")
                            rail_flag = True
                            break
                    
            """以下のfor文は「path[1]が見つかった場合のみ回る」、例えばvalues=小76／松本橋東詰6分　の時はstationに何も加えない"""
            sta_flag = False
            for eki in all_stations:
                for val in values.split(path[1],1)[1:]:
                    if eki[1] in val and sta_flag == False:
                        station.append(eki[1])
                        sta_flag = True
                        break
                        
                    if eki[1] == all_stations[-1][1]:
                        if sta_flag == False:   #Trueにしたら新規の駅が除かれる
                            for tmp_eki in all_stations:
                                if tmp_eki[1] == values.split(path[1],1)[1].split("駅")[0]+"(東京都)駅":
                                    station.append(tmp_eki[1])
                                    eki[1] = values.split(path[1],1)[1].split("駅")[0]+"駅"
                                    sta_flag = True
                                    break
                            if sta_flag == False:
                                station.append("NaN")
                                sta_flag = True
                                break
                        
                if sta_flag:
                    break
            
            if len(values.split(eki[1])[1:])==0:   # 駅や路線が新たに追加された時用  #小76／松本橋東詰6分 三田線　未処理
                try:
                    #print(values, eki[1], path[1] , "  ", values.split(path[1],1)[1].split("駅")[0]+"駅")
                    new_station.append(values.split(path[1],1)[1].split("駅")[0]+"駅")
                except:
                    print(values)
            search_len.append(len(values.split(eki[1])))

            
            try:
                if "km" not in values.split(path[1],1)[1].split(eki[1],1)[1] and "分" not in values.split(path[1],1)[1].split(eki[1],1)[1]:
                    minute.append(np.nan)
                    print(values)
                else:    
                    meter_infos = re.split("分", values.split(path[1],1)[1].split(eki[1],1)[1])
                    minute.append(calculate_meter(meter_infos))
            
            except:
                #print(values, eki[1], path[1])
                if sta_flag == False:
                    station.append("NaN")
                minute.append(np.nan)
            
            """if values == "小76／松本橋東詰6分":
                station.append(1)
                railway.append(2)"""
            

        """ i==1のなかにi>=2の文が詰め込まれているパターン"""
        """if len(values.split("\t")) > 3:
            print(i, values.split("\t"))
            railway.append(values.split("\t")[0])
            station.append(values.split("\t")[1])
            tmp = values.split("\t")[2:]
            for t in tmp:
                if t in all_railways and i < 3:
                    df_out[f"路線_{i+1}"] = t
                    print(tmp.split(t)[0])
                    minute.append(calculate_meter(tmp.split(t)[0]))
                else:
                    minute.append(calculate_meter(tmp))"""
        
    df_out[f"路線_{i}"] = railway
    df_out[f"駅_{i}"] = station
    df_out[f"駅からの距離_{i}"] = minute
    print(f"{i}_done")

小76／松本橋東詰6分
1_done
亀29／新田4分
新小21／棒茅場6分
亀26／東小松川一2分
船28／新堀一3分
小72／東小岩六5分
新小22／江戸川区口腔保健センター入口4分
京成電鉄本線京成小岩駅徒歩16分
新小21／新小岩一6分
亀29／堀江団地1分
葛西21／第二葛西小入口3分
西葛26／南葛西中4分
葛西21／南葛西中4分
葛西21／東葛西六4分
新小21／松江第一中5分
001／葛西駅4分
東22／江東区役所3分
葛西22／浦安橋5分
武蔵野線新座駅/バス(20分)久保新田下車徒歩2分
東京メトロ丸ノ内線荻窪駅徒歩24分
関東バス／善福寺緑地公園7分
臨海28乙／葛西南高校入口5分
東京メトロ丸ノ内線四谷三丁目駅徒歩12分
東京メトロ丸ノ内線茗荷谷駅徒歩14分
小76／鹿骨四3分
東22／江東区役所4分
001／葛西駅5分
北陸新幹線(東京－上越妙高)上野駅徒歩15分
臨海28乙／仲町西組1分
新小21／松江第一中5分
東京メトロ丸ノ内線新中野駅徒歩12分
小76／鹿骨四5分
竹03／氷川神社前4分
新小22／江戸川区口腔保健センター入口4分
武蔵野線三郷(埼玉県)駅/バス(35分)桜土手下車徒歩3分
東京メトロ丸ノ内線荻窪駅徒歩18分
日暮里舎人ライナー江北駅徒歩15分
東武伊勢崎・大師線牛田駅徒歩3分
渋24／農大一高前2分
001／葛西駅3分
東武伊勢崎・大師線堀切駅徒歩22分
亀29／南葛西小2分
新小21／七軒町1分
東京メトロ丸ノ内線淡路町駅徒歩7分
葛西21／南葛西会館2分
関東バス／成田東三丁目2分
椿04／江北三1分
東西線落合駅
東京メトロ丸ノ内線新中野駅徒歩7分
亀29／西葛西六5分
葛西22／浦安橋3分
環07急行／総合レクリエーション公園6分
亀29／なぎさニュータウン3分
東京メトロ丸ノ内線四ツ谷駅徒歩7分
小76／明和橋2分
葛西21／南葛西中2分
葛西21／南葛西中4分
新小30／葛西第三中入口3分
篠01／鹿骨一公園2分
中24／江古田三3分
葛西21／第二葛西小入口5分
渋24／大蔵ランド前3分
新小21／棒茅場2分
亀29／中葛西五5分
葛西21／南葛西中2分
新小21／東小松川二5分
小74／京葉交差点3分
見01／見沼代親水公園駅5分
新小20／小松中3分
都営バス／堀ノ内2分
葛西21／東葛西七3分


In [7]:
for i in range(0,3):
    idx = df_out[f"路線_{i}"] == "三田線"
    df_out[f"路線_{i}"][idx] = "都営三田線"

    idx = df_out[f"路線_{i}"] == "中央・総武緩行線"
    df_out[f"路線_{i}"][idx] = "総武線・中央線（各停）"
    
    idx = df_out[f"路線_{i}"] == "中央本線"
    df_out[f"路線_{i}"][idx] = '中央本線(東京－松本)'
    
    idx = df_out[f"路線_{i}"] == "東北新幹線"
    df_out[f"路線_{i}"][idx] = "山手線"
    
    idx = df_out[f"路線_{i}"] == "京葉線"
    df_out[f"路線_{i}"][idx] = "京葉線(東京－蘇我)"
    
    idx = df_out[f"駅_{i}"] == "三田(兵庫県)駅"
    df_out[f"駅_{i}"][idx] = "三田(東京都)駅"

/Users/ueda/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/ueda/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/ueda/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

In [8]:
mixed_df = pd.DataFrame(columns = ["路線", "駅", "距離"])

mixed_df["路線"] = pd.concat([df_out["路線_0"], df_out["路線_1"], df_out["路線_2"]])
mixed_df["駅"] = pd.concat([df_out["駅_0"], df_out["駅_1"], df_out["駅_2"]])
mixed_df["距離"] = pd.concat([df_out["駅からの距離_0"], df_out["駅からの距離_1"], df_out["駅からの距離_2"]])

In [9]:
stats = ["mean", "std", "min", "max"]
columns = ["駅_0", "駅_1", "駅_2"]
for col in columns:
    for agg in stats:
        temp = mixed_df.groupby('駅')['距離'].agg([agg]).rename({agg:f'{col}からの距離_{agg}'}, axis=1)
        temp.index.name = f"{col}"
        df_out = pd.merge(df_out,temp,on=f'{col}',how='left')

"""importance　0"""        
#columns = ["路線_0", "路線_1", "路線_2"]
#for col in columns:
#    for agg in stats:
#        temp = mixed_df.groupby('駅')['距離'].agg([agg]).rename({agg:f'{col}からの距離_{agg}'}, axis=1)
#        temp.index.name = f"{col}"
#        df_out = pd.merge(df_out,temp,on=f'{col}',how='left')

'importance\u30000'

In [10]:
df_out["路線_0_counts"] = df_out["路線_0"].map(mixed_df["路線"].value_counts().to_dict())
df_out["路線_1_counts"] = df_out["路線_1"].map(mixed_df["路線"].value_counts().to_dict())
df_out["路線_2_counts"] = df_out["路線_2"].map(mixed_df["路線"].value_counts().to_dict())

df_out["駅_0_counts"] = df_out["駅_0"].map(mixed_df["駅"].value_counts().to_dict())
df_out["駅_1_counts"] = df_out["駅_1"].map(mixed_df["駅"].value_counts().to_dict())
df_out["駅_2_counts"] = df_out["駅_2"].map(mixed_df["駅"].value_counts().to_dict())

In [11]:
df_out["路線_counts_max"] = df_out[["路線_0_counts", "路線_1_counts", "路線_2_counts"]].max(axis=1)
df_out["路線_counts_mean"] = df_out[["路線_0_counts", "路線_1_counts", "路線_2_counts"]].mean(axis=1)
df_out["路線_counts_min"] = df_out[["路線_0_counts", "路線_1_counts", "路線_2_counts"]].min(axis=1)
df_out["路線_counts_std"] = df_out[["路線_0_counts", "路線_1_counts", "路線_2_counts"]].std(axis=1)

df_out["駅_counts_max"] = df_out[["駅_0_counts", "駅_1_counts", "駅_2_counts"]].max(axis=1)
df_out["駅_counts_mean"] = df_out[["駅_0_counts", "駅_1_counts", "駅_2_counts"]].mean(axis=1)
df_out["駅_counts_min"] = df_out[["駅_0_counts", "駅_1_counts", "駅_2_counts"]].min(axis=1)
df_out["駅_counts_std"] = df_out[["駅_0_counts", "駅_1_counts", "駅_2_counts"]].std(axis=1)

df_out["距離_max"] = df_out[["駅からの距離_0", "駅からの距離_1", "駅からの距離_2"]].max(axis=1)
df_out["距離_min"] = df_out[["駅からの距離_0", "駅からの距離_1", "駅からの距離_2"]].min(axis=1)
df_out["距離_mean"] = df_out[["駅からの距離_0", "駅からの距離_1", "駅からの距離_2"]].mean(axis=1)
df_out["距離_std"] = df_out[["駅からの距離_0", "駅からの距離_1", "駅からの距離_2"]].std(axis=1)

df_out["路線_0_counts/距離"] = df_out["路線_0_counts"] / df_out["駅からの距離_0"]
df_out["路線_1_counts/距離"] = df_out["路線_1_counts"] / df_out["駅からの距離_1"]
df_out["路線_2_counts/距離"] = df_out["路線_2_counts"] / df_out["駅からの距離_2"]

df_out["駅_0_counts/距離"] = df_out["駅_0_counts"] / df_out["駅からの距離_0"]
df_out["駅_1_counts/距離"] = df_out["駅_1_counts"] / df_out["駅からの距離_1"]
df_out["駅_2_counts/距離"] = df_out["駅_2_counts"] / df_out["駅からの距離_2"]

df_out["路線counts/距離_max"] = df_out[["路線_0_counts/距離", "路線_1_counts/距離", "路線_2_counts/距離"]].max(axis=1)
df_out["路線counts/距離_mean"] = df_out[["路線_0_counts/距離", "路線_1_counts/距離", "路線_2_counts/距離"]].mean(axis=1)
df_out["路線counts/距離_min"] = df_out[["路線_0_counts/距離", "路線_1_counts/距離", "路線_2_counts/距離"]].min(axis=1)
df_out["路線counts/距離_std"] = df_out[["路線_0_counts/距離", "路線_1_counts/距離", "路線_2_counts/距離"]].std(axis=1)

df_out["駅counts/距離_max"] = df_out[["駅_0_counts/距離", "駅_1_counts/距離", "駅_2_counts/距離"]].max(axis=1)
df_out["駅counts/距離_mean"] = df_out[["駅_0_counts/距離", "駅_1_counts/距離", "駅_2_counts/距離"]].mean(axis=1)
df_out["駅counts/距離_min"] = df_out[["駅_0_counts/距離", "駅_1_counts/距離", "駅_2_counts/距離"]].min(axis=1)
df_out["駅counts/距離_std"] = df_out[["駅_0_counts/距離", "駅_1_counts/距離", "駅_2_counts/距離"]].std(axis=1)

In [12]:
df_out["駅_0からの距離_mean_diff"] = df_out["駅_0からの距離_mean"] - df_out["駅からの距離_0"]
df_out["駅_1からの距離_mean_diff"] = df_out["駅_1からの距離_mean"] - df_out["駅からの距離_1"]
df_out["駅_2からの距離_mean_diff"] = df_out["駅_2からの距離_mean"] - df_out["駅からの距離_2"]

In [13]:
"""アクセス方法の記載量"""
output = np.full_like(df_out["路線_0"], 3)
idx2 = tmp_df[2] == "NaN"
output[idx2] = 2
idx1 = tmp_df[1] == "NaN"
output[idx1] = 1
df_out["access_way_count"] = output

In [14]:
"""位置情報"""
"""
ido_kedo_dict = {}
for station_name in [ i for i in mixed_df["駅"].value_counts().to_dict().keys() if i != "NaN"]:
    ret = geocoder.osm(station_name, timeout=5.0)
    try:
        ido, kedo = ret.latlng
        if int(ido) != 35 or int(kedo) != 139:
            ido_kedo_dict[station_name] = -999
        else:    
            ido_kedo_dict[station_name] = (ido, kedo)
    except:
        ido_kedo_dict[station_name] = -999"""

'\nido_kedo_dict = {}\nfor station_name in [ i for i in mixed_df["駅"].value_counts().to_dict().keys() if i != "NaN"]:\n    ret = geocoder.osm(station_name, timeout=5.0)\n    try:\n        ido, kedo = ret.latlng\n        if int(ido) != 35 or int(kedo) != 139:\n            ido_kedo_dict[station_name] = -999\n        else:    \n            ido_kedo_dict[station_name] = (ido, kedo)\n    except:\n        ido_kedo_dict[station_name] = -999'

In [15]:
""" -999 の処理をいつか書く """

' -999 の処理をいつか書く '

In [16]:
"""feather output"""
"""
temp = pd.DataFrame(ido_kedo_dict).T
temp = temp.reset_index()
temp.columns = ["駅名", "駅緯度", "駅経度"]
temp.to_feather("../input/eki_ido_kedo.feather")"""

'\ntemp = pd.DataFrame(ido_kedo_dict).T\ntemp = temp.reset_index()\ntemp.columns = ["駅名", "駅緯度", "駅経度"]\ntemp.to_feather("../input/eki_ido_kedo.feather")'

In [17]:
df_out[["駅からの距離_0", "駅からの距離_1", "駅からの距離_2"]].describe()

,駅からの距離_0,駅からの距離_1,駅からの距離_2
count,62732.000000,59747.000000,56117.000000
mean,642.244469,1057.015415,1392.325320
std,570.805374,920.751213,1204.006771
min,80.000000,80.000000,80.000000
25%,320.000000,640.000000,880.000000
50%,560.000000,880.000000,1200.000000
75%,800.000000,1200.000000,1600.000000
max,10960.000000,24240.000000,25600.000000


In [18]:
a = feather.read_dataframe("../input/eki_ido_kedo.feather")
b = feather.read_dataframe("../input/eki_ido_kedo_nokori.feather")

idx = a["駅緯度"] == -999
idxx = a["駅緯度"] != -999

s = a[idxx]

b_dict = b.set_index("駅名").to_dict()
eki_ido_kedo = a
eki_ido_kedo["駅緯度"] = a[idx]["駅名"].map(b_dict["駅緯度"])
eki_ido_kedo["駅経度"] = a[idx]["駅名"].map(b_dict["駅経度"])
eki_ido_kedo[idxx] = s
eki_ido_kedo[["駅緯度", "駅経度"]] = eki_ido_kedo[["駅緯度", "駅経度"]].astype(float)

comp_dict = eki_ido_kedo.set_index("駅名").to_dict()


columns = ['駅_0', '駅_1', '駅_2']
for column in columns:
    df_out[f"{column}_緯度"] = df_out[column].map(comp_dict["駅緯度"])
    df_out[f"{column}_経度"] = df_out[column].map(comp_dict["駅経度"])
df_out.describe()

,駅からの距離_0,駅からの距離_1,駅からの距離_2,駅_0からの距離_mean,駅_0からの距離_std,駅_0からの距離_min,駅_0からの距離_max,駅_1からの距離_mean,駅_1からの距離_std,駅_1からの距離_min,...,駅counts/距離_std,駅_0からの距離_mean_diff,駅_1からの距離_mean_diff,駅_2からの距離_mean_diff,駅_0_緯度,駅_0_経度,駅_1_緯度,駅_1_経度,駅_2_緯度,駅_2_経度
count,62732.000000,59747.000000,56117.000000,62732.000000,62730.000000,62732.000000,62732.000000,59747.000000,59745.000000,59747.000000,...,59747.000000,62732.000000,59747.000000,56117.000000,62732.000000,62732.000000,59747.000000,59747.000000,56118.000000,56118.000000
mean,642.244469,1057.015415,1392.325320,1027.759624,708.999430,98.020787,5983.633233,1006.042282,621.259119,108.532311,...,0.599268,385.515156,-50.973133,-376.688793,35.693359,139.729745,35.692794,139.728134,35.692441,139.726749
std,570.805374,920.751213,1204.006771,440.995422,568.599202,69.843461,5314.867167,482.329520,523.451524,130.871460,...,0.782183,544.716729,771.414811,1001.922620,0.055540,0.076538,0.055204,0.079051,0.054945,0.080218
min,80.000000,80.000000,80.000000,334.344828,113.137085,80.000000,720.000000,334.344828,78.655366,80.000000,...,0.000000,-8870.490566,-20495.960591,-21019.653179,35.531178,139.007360,35.531178,139.007360,35.531178,139.007360
25%,320.000000,640.000000,880.000000,766.866485,373.121138,80.000000,2000.000000,771.346705,350.166008,80.000000,...,0.189568,163.021869,-232.181303,-534.059946,35.658732,139.672922,35.658242,139.673290,35.658242,139.676475
50%,560.000000,880.000000,1200.000000,935.532646,477.276709,80.000000,2720.000000,918.087954,445.115716,80.000000,...,0.359799,379.810726,14.726989,-268.452656,35.697881,139.724179,35.697462,139.723497,35.696908,139.723497
75%,800.000000,1200.000000,1600.000000,1128.098765,761.223603,80.000000,9680.000000,1098.288100,636.655798,80.000000,...,0.683374,608.822554,234.749035,-19.417476,35.733715,139.784563,35.735009,139.783536,35.733704,139.781463
max,10960.000000,24240.000000,25600.000000,7330.500000,5931.688879,2960.000000,25600.000000,7330.500000,5931.688879,6880.000000,...,9.546872,5828.456376,5668.456376,5508.456376,35.845811,139.928942,35.822753,139.928942,35.824521,139.928942


In [19]:
x = df_out["駅_0"].value_counts() < 30

new_eki_list = []
for t in df_out["駅_0"]:
    if x[t] == True:
        new_eki_list.append("minority_sta")
    else:
        new_eki_list.append(t)
df_out["catuse_駅_0"] = new_eki_list

In [20]:
y = df_out["路線_0"].value_counts() < 30

new_railway_list = []
for t in df_out["路線_0"]:
    if y[t] == True:
        new_railway_list.append("minority_rail")
    else:
        new_railway_list.append(t)
df_out["catuse_路線_0"] = new_railway_list

In [33]:
df_out["路線_0"].value_counts().to_dict()

{'山手線': 5112,
 '総武線・中央線（各停）': 2806,
 '都営大江戸線': 2716,
 '東西線': 2693,
 '都営三田線': 2609,
 '西武池袋線': 2441,
 '有楽町線': 2309,
 '都営新宿線': 2268,
 '京浜東北線': 2048,
 '日比谷線': 1970,
 '京王線': 1960,
 '東急田園都市線': 1948,
 '千代田線': 1895,
 '丸ノ内線(池袋－荻窪)': 1855,
 '東武スカイツリーライン': 1698,
 '東武東上線': 1653,
 '中央線（快速）': 1602,
 '小田急小田原線': 1578,
 '都営浅草線': 1578,
 '西武新宿線': 1521,
 '東急東横線': 1432,
 '南北線': 1357,
 '京急本線': 1236,
 '半蔵門線': 1127,
 '東急池上線': 1064,
 '常磐線': 1057,
 '京王井の頭線': 1009,
 '日暮里・舎人ライナー': 876,
 '京成本線': 839,
 '東急目黒線': 826,
 '東急大井町線': 732,
 '丸ノ内線(中野坂上－方南町)': 644,
 'つくばエクスプレス': 627,
 '銀座線': 617,
 '埼京線': 616,
 '京王新線': 533,
 '京成押上線': 480,
 '総武本線': 415,
 '東急多摩川線': 387,
 '副都心線': 379,
 '東急世田谷線': 361,
 '京急空港線': 329,
 '常磐線快速': 184,
 '京葉線(東京－蘇我)': 184,
 'ゆりかもめ': 140,
 '京成金町線': 131,
 '横須賀線': 122,
 'りんかい線': 121,
 '東武亀戸線': 115,
 '都電荒川線': 102,
 '東武大師線': 77,
 '西武有楽町線': 77,
 '高崎線': 75,
 '東京モノレール羽田線': 44,
 '北総線': 38,
 '東武伊勢崎線(押上－曳舟)': 38,
 '東海道新幹線': 24,
 '西武豊島線': 23,
 '東北本線（宇都宮線）': 12,
 '東海道本線': 12,
 '湘南新宿ライン(東北本線－横須賀線)': 5,
 '中央本線(東京－松本)

In [21]:
"""ファイル出力"""
all_df_feature = df_out
output_dir = '../code/feature_csv/'
all_df_feature.to_feather(os.path.join(output_dir, 'access_feature.feather'))

In [27]:
def station_hash_feature(df_input):
    z = mixed_df["駅"].value_counts() > 20
    all_majoeki_list = list(mixed_df["駅"].value_counts().keys())
    all_majoeki_list.remove("NaN")
    
    df_out = pd.DataFrame()
    for eki in all_majoeki_list:
        if z[eki]==True:
            tmp_array = np.where(df_input["駅_0"]== eki, 1, 0)
            tmp_array1 = np.where(df_input["駅_1"]== eki, 1, 0)
            tmp_array2 = np.where(df_input["駅_2"]== eki, 1, 0)
            last_array = tmp_array + tmp_array1 + tmp_array2
            p = np.where(last_array > 1, 1, last_array)
            df_out[f"near_{eki}"] = p
        else:
            continue
    return df_out

In [28]:
#df_out = pd.concat([df_out, station_hash_feature(df_out)], axis = 1)
df_near = station_hash_feature(df_out)

In [29]:
df_near.columns

Index(['near_池袋駅', 'near_綾瀬駅', 'near_三軒茶屋駅', 'near_錦糸町駅', 'near_葛西駅',
       'near_蒲田駅', 'near_小岩駅', 'near_門前仲町駅', 'near_月島駅', 'near_中野(東京都)駅',
       ...
       'near_新宿西口駅', 'near_荒川遊園地前駅', 'near_有明(東京都)駅', 'near_港町駅',
       'near_荒川車庫前駅', 'near_飛鳥山駅', 'near_日暮里(舎人ライナー)駅', 'near_東池袋四丁目駅',
       'near_市川駅', 'near_鬼子母神前駅'],
      dtype='object', length=488)

In [30]:
import sys
sys.path.append("/Users/ueda/Desktop/kaggle/mynavi/code/src/")
from util_tool import reduce_mem_usage
df_near = reduce_mem_usage(df_near)

Memory usage of dataframe is 233.56 MB
column =  488
0
50
100
150
200
250
300
350
400
450
Memory usage after optimization is: 29.20 MB
Decreased by 87.5%


In [31]:
"""ファイル出力"""
all_df_feature = df_near
output_dir = '../code/feature_csv/'
all_df_feature.to_feather(os.path.join(output_dir, 'near_access_feature.feather'))

In [21]:
print(len(railway), len(station), len(minute))

62732 62732 62732


In [22]:
print(list(set(new_station)))

['六本木徒歩10分駅', '恵比寿徒歩5分駅', '竹芝駅', '浜松町駅', '本蓮沼駅', '豊島園駅', '三田駅', '品川駅', '面影橋駅', '新江古田駅', '田無駅', '新宿(東京メトロ)駅', '・要町駅', '荏原中延駅', '蒲田駅', '浦和駅', '向河原駅', '汐留駅', '上野広小路徒歩8分・山手線上野徒歩10分駅', '下丸子駅', '矢口渡駅', '大岡山駅', '若松河田駅', '門前仲町駅', '東長崎駅', '富士見ヶ丘駅', '向原駅', '大門駅', '池袋駅', '小竹向原駅', '市ヶ谷駅', '下板橋駅', '鬼子母神前停徒歩３分駅', '早稲田駅', '新橋駅', '下赤塚駅', '戸田公園駅']


In [23]:
print(all_railways)

[[18, '湘南新宿ライン(高崎線－東海道本線)'], [18, '湘南新宿ライン(東北本線－横須賀線)'], [14, '丸ノ内線(中野坂上－方南町)'], [13, '東武伊勢崎線(押上－曳舟)'], [12, 'ディズニーリゾートライン'], [11, '総武線・中央線（各停）'], [11, '東武スカイツリーライン'], [11, '丸ノ内線(池袋－荻窪)'], [11, '中央本線(東京－松本)'], [10, '東北本線（宇都宮線）'], [10, '東京モノレール羽田線'], [10, '日暮里・舎人ライナー'], [10, '南武線(川崎－立川)'], [10, '京葉線(東京－蘇我)'], [9, 'つくばエクスプレス'], [7, '東急田園都市線'], [7, '小田急小田原線'], [7, '埼玉高速鉄道線'], [7, '京成成田空港線'], [7, '中央線（快速）'], [6, '都営大江戸線'], [6, '西武有楽町線'], [6, '東海道新幹線'], [6, '東急大井町線'], [6, '東急多摩川線'], [6, '東急世田谷線'], [6, '京王井の頭線'], [5, '都電荒川線'], [5, '都営浅草線'], [5, '都営新宿線'], [5, '都営三田線'], [5, '西武豊島線'], [5, '西武池袋線'], [5, '西武新宿線'], [5, '東海道本線'], [5, '東武東上線'], [5, '東武大師線'], [5, '東武亀戸線'], [5, '東急目黒線'], [5, '東急池上線'], [5, '東急東横線'], [5, '東北新幹線'], [5, '常磐線快速'], [5, '京浜東北線'], [5, '京成金町線'], [5, '京成押上線'], [5, '京急空港線'], [5, '京急大師線'], [5, 'りんかい線'], [5, 'ゆりかもめ'], [4, '総武本線'], [4, '横須賀線'], [4, '有楽町線'], [4, '日比谷線'], [4, '半蔵門線'], [4, '千代田線'], [4, '副都心線'], [4, '京王新線'], [4, '京成本線'], [4, '京急本線'], [3, '高崎線'], [3, '銀座線'], [3, '東西線'],